In [ ]:
# Load the python package
import os
# import dynetan
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

#from numpy.linalg import norm
from itertools import islice
from scipy import stats

import pandas as pd
import numpy as np
import scipy as sp

In [ ]:
dnad = DNAdata()

# Load network data obtained by Dynetan module 

In [ ]:
dataDir = "/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA"

# Path where results will be written (you may want plots and data files in a new location)
workDir = "/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA"

fileNameRoot = "crRNA_network"
fullPathRoot = os.path.join(dataDir, fileNameRoot)

# Define the segID of the Ligand being studied.
ligandSegID = "SYST"
dnad.loadFromFile(fullPathRoot)
dcdVizFile = fullPathRoot + "_reducedTraj.dcd"
pdbVizFile = fullPathRoot + "_reducedTraj.pdb"

workUviz = mda.Universe(pdbVizFile, dcdVizFile)
# We add this to the object for ease of access.
dnad.nodesAtmSel = workUviz.atoms[ dnad.nodesIxArray ]
print(dnad.nodesAtmSel)

A function to calculate pathlengths in terms of edge betweenness:

In [ ]:
def getBCsum(List, Indx):
        bc = 0
        # Iterate over edges in the path
        for i in range(len(List)-1):
            node1 = List[i]
            node2 = List[i+1]
            if node1 > node2:
                btw = (dnad.btws[Indx][( node2, node1)])/maximumBetweeness
            else:
                btw = (dnad.btws[Indx][( node1, node2)])/maximumBetweeness
            bc += btw
        return bc

Estimate of Noise:

In [ ]:
import copy
source = list(range(1153,1203)) ## crRNA bases
sink = list(range(0,1153)) ## Cas13 residues

# Initialize variable with high value.
minimumBetweeness = 100
# Initialize variable with low value.
maximumBetweeness = -1

for pair,btw in dnad.btws[0].items():
    if btw < minimumBetweeness:
            minimumBetweeness = btw
    if btw > maximumBetweeness:
            maximumBetweeness = btw

# Normalize the value.
minimumBetweeness /= maximumBetweeness
# Determine how many extra sub-optimal paths will be written.
numSuboptimalPaths = 5   
        
pathListFileA = open(os.path.join(workDir, "BCSum_pathlength6-10.dat"), "w")
pathListFileB = open(os.path.join(workDir, "BCSum_pathlength11-14.dat"), "w")

for srcNode in source:
    print(srcNode)
    for trgNode in sink:

        tmpList = getSelFromNode(srcNode,dnad.nodesAtmSel, atom=True).split()
        srcNodeSel = "".join([tmpList[1],tmpList[4],tmpList[10]])

        tmpList = getSelFromNode(trgNode,dnad.nodesAtmSel, atom=True).split()
        trgNodeSel = "".join([tmpList[1],tmpList[4],tmpList[10]])

        for winIndx in range(dnad.numWinds):

            # Reconstructs the optimal path from Floyd-Warshall algorithm
            pathFW = nx.reconstruct_path(srcNode, trgNode, dnad.preds[winIndx])
#             print(pathFW)
            ### ---- Get BCSum for optimal paths of lengths 6 - 10 & 11 - 14 ---- ####
            if ((len(pathFW) < 11) and (len(pathFW) >= 6)):
                BC = getBCsum(pathFW, winIndx)
                pathListFileA.write(str(BC) + "\n")
                
            if ((len(pathFW) <= 14) and (len(pathFW) >= 11)):
                BC = getBCsum(pathFW, winIndx)
                pathListFileB.write(str(BC) + "\n")

#             Behind the scenes, use Dijkstra algorithm to find sub-optimal paths
            for pathSO in islice(nx.shortest_simple_paths(dnad.nxGraphs[0], 
                                                srcNode, trgNode, weight="dist"), 1, numSuboptimalPaths + 1):
             ### ---- Get BCSum for sub-optimal paths of lengths 6 - 10 & 11 - 14 ---- ####
                if ((len(pathSO) < 11) and (len(pathSO) >= 6)):
                    BC = getBCsum(pathSO, winIndx)
                    pathListFileA.write(str(BC) + "\n")
                    
                if ((len(pathSO) <= 14) and (len(pathSO) >= 11)):
                    BC = getBCsum(pathSO, winIndx)
                    pathListFileB.write(str(BC) + "\n")

pathListFileA.close()
pathListFileB.close()
         

Estimate of Signals and write down the paths:

In [ ]:
source = list(range(1183,1187)) ## Specific crRNA bases correspond to spacer
sink = [1047, 1052, 471, 476] ## Catalytic residues

# Initialize variable with high value.
minimumBetweeness = 100
# Initialize variable with low value.
maximumBetweeness = -1

for pair,btw in dnad.btws[0].items():
    if btw < minimumBetweeness:
            minimumBetweeness = btw
    if btw > maximumBetweeness:
            maximumBetweeness = btw

# Normalize the value.
minimumBetweeness /= maximumBetweeness
# Determine how many extra sub-optimal paths will be written.
numSuboptimalPaths = 5   

pathListFileA = open(os.path.join(workDir, "BCSum_sub&optlength6-10_initTocatres.dat"), "w")
pathListFileB = open(os.path.join(workDir, "BCSum_sub&optlength11-14_initTocatres.dat"), "w")
pathListFileC = open(os.path.join(workDir, "initTocatres_allpaths.dat"), "w")

for srcNode in source:
    print(srcNode)
    for trgNode in sink:

        tmpList = getSelFromNode(srcNode,dnad.nodesAtmSel, atom=True).split()
        srcNodeSel = "".join([tmpList[1],tmpList[4],tmpList[10]])

        tmpList = getSelFromNode(trgNode,dnad.nodesAtmSel, atom=True).split()
        trgNodeSel = "".join([tmpList[1],tmpList[4],tmpList[10]])

        for winIndx in range(dnad.numWinds):
            
            normCorMat = copy.deepcopy( dnad.corrMatAll[winIndx,:,:] )
            normCorMat /= normCorMat.max()
            
            allPaths = []
            
            # Reconstructs the optimal path from Floyd-Warshall algorithm
            pathFW = nx.reconstruct_path(srcNode, trgNode, dnad.preds[winIndx])
            allPaths.append(pathFW)
            ### ---- Get BCSum for optimal paths of lengths 6 - 10 & 11 - 14 ---- ####
            if ((len(pathFW) < 11) and (len(pathFW) >= 6)):
                BC = getBCsum(pathFW, winIndx)
                pathListFileA.write(str(BC) + "\n")

            if ((len(pathFW) <= 14) and (len(pathFW) >= 11)):
                BC = getBCsum(pathFW, winIndx)
                pathListFileB.write(str(BC) + "\n")

#             Behind the scenes, use Dijkstra algorithm to find sub-optimal paths
            for pathSO in islice(nx.shortest_simple_paths(dnad.nxGraphs[0], 
                                                srcNode, trgNode, weight="dist"), 1, numSuboptimalPaths + 1):
                allPaths.append(pathSO)
            ### ---- Get BCSum for sub-optimal paths of lengths 6 - 10 & 11 - 14 ---- ####
                if ((len(pathSO) < 11) and (len(pathSO) >= 6)):
                    BC = getBCsum(pathSO, winIndx)
                    pathListFileA.write(str(BC) + "\n")

                if ((len(pathSO) <= 14) and (len(pathSO) >= 11)):
                    BC = getBCsum(pathSO, winIndx)
                    pathListFileB.write(str(BC) + "\n")
                    
            ### ---- Write all the paths calculated so far ---- ####      
            
         # Create a counter of number of paths that go though each edge, among all (sub-)optimal path(s).
            pathCounter = defaultdict(int)
            for pathIndx, pathIter in enumerate(allPaths):
                # Iterate over edges in the path
                for i in range(len(pathIter)-1):

                    node1 = pathIter[i]
                    node2 = pathIter[i+1]

                    pathCounter[(node1, node2)] += 1
#                 print(pathCounter)

            # Normalize the count
            maxCount = np.max(list(pathCounter.values()))
            for pair, count in pathCounter.items():
                pathCounter[pair] = count/maxCount

            for pathIndx, pathIter in enumerate(allPaths):
            # Iterate over edges in the path
                for i in range(len(pathIter)-1):

                    node1 = pathIter[i] + 1
                    node2 = pathIter[i+1] + 1

                     # Get the betweeness value
                    try:
                        if node1 > node2:
                            btw = (dnad.btws[winIndx][( node2, node1)])
                        else:
                            btw = (dnad.btws[winIndx][( node1, node2)])
                    except:
                        # If one could not be calculated (very few paths going though this edge)
                         # set an arbitrarily low value.
                        btw = minimumBetweeness


                    string = "{} {} {} {} {} {}".format(node1, node2, 
                                                         normCorMat[ node1, node2], 
                                                         btw/maximumBetweeness, pathCounter[(node1, node2)], 
                                                         pathIndx)

                    pathListFileC.write( string + "\n" )


pathListFileA.close()
pathListFileB.close()
pathListFileC.close()

## Plotting and Calculating SNR

Load Noise and Signal data:

In [ ]:
import numpy as np
cr_6_10 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_pathlength6-10.dat')
cr_11_14 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_pathlength11-14.dat')

cr_6_10_itoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength6-10_initTocatres.dat')
cr_11_14_itoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength11-14_initTocatres.dat')

cr_6_10_swtoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength6-10_switchTocatres.dat')
cr_11_14_swtoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength11-14_switchTocatres.dat')

cr_6_10_setoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength6-10_seedTocatres.dat')
cr_11_14_setoc = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/BCSum_sub&optlength11-14_seedTocatres.dat')



Plot:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
## Plotting
%matplotlib inline
fig = plt.figure(figsize=(8,14))
spec = gridspec.GridSpec(ncols=1, nrows=2, hspace=0.3, figure=fig)
ax1 = fig.add_subplot(spec[0, 0])
ax2 = fig.add_subplot(spec[1, 0])

import seaborn as sb

sb.kdeplot(cr_6_10, color="grey", shade=True, alpha=0.5, ax=ax1, linewidth=1, label = 'crRNA')
sb.kdeplot(cr_11_14, color="grey", shade=True, alpha=0.5, ax=ax2, linewidth=1, label = 'crRNA')
 
ax1.set_ylim([0, 1.5])
ax1.set_xlim([-1, 5])
ax1.set_title("Betweenness Distribution of path length 6 to 10", fontsize=16, fontweight='bold')
ax1.scatter(cr_6_10_itoc,[0.40] * cr_6_10_itoc.shape[0],marker = "o", c="red")
ax1.scatter(cr_6_10_swtoc,[0.80] * cr_6_10_swtoc.shape[0],marker = "o", c="green")
ax1.scatter(cr_6_10_setoc,[1.20] * cr_6_10_setoc.shape[0],marker = "o", c="blue")
ax1.set_ylabel('Density', fontsize=14, fontweight='bold')

ax2.set_ylim([0, 1.5])
ax2.set_xlim([-1, 5])
ax2.set_title("Betweenness Distribution of path length 11 to 14", fontsize=16, fontweight='bold')
ax2.scatter(cr_11_14_swtoc,[0.80] * cr_11_14_swtoc.shape[0],marker = "o", c="green")
ax2.scatter(cr_11_14_setoc,[1.20] * cr_11_14_setoc.shape[0],marker = "o", c="blue")
ax2.set_ylabel('Density', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig("/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/SNR_Distribution_V2.png", dpi=600)

# Calculate SNR

In [ ]:
import numpy as np
from scipy.integrate import quad
from sklearn.neighbors import KernelDensity

def expectation(KDE):
    pdf = lambda x : np.exp(KDE.score_samples([[x]]))[0]
    mean_integration = quad(lambda x: x * pdf(x), a=-np.inf, b=np.inf)[0]
    variance_integration = quad(lambda x: (x ** 2) * pdf(x), a=-np.inf, b=np.inf)[0] - mean_integration ** 2
    return mean_integration/variance_integration

    


kde_cr_11_14 = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(cr_11_14[:, np.newaxis])
# kde_cr_11_14_itoc = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(cr_11_14_itoc[:, np.newaxis])
kde_cr_11_14_swtoc = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(cr_11_14_swtoc[:, np.newaxis])
kde_cr_11_14_setoc = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(cr_11_14_setoc[:, np.newaxis])


print("Expectation of cr_11_14 noise:", expectation(kde_cr_11_14))

# print("Expectation of cr_11_14_itoc noise:", expectation(kde_cr_11_14_itoc))

print("Expectation of cr_11_14_swtoc noise:", expectation(kde_cr_11_14_swtoc))

print("Expectation of cr_11_14_setoc noise:", expectation(kde_cr_11_14_setoc))

# print("SNR cr_11_14_itoc:", expectation(kde_cr_11_14_itoc)/expectation(kde_cr_11_14))

print("SNR cr_11_14_swtoc:", expectation(kde_cr_11_14_swtoc)/expectation(kde_cr_11_14))

print("SNR cr_11_14_setoc:", expectation(kde_cr_11_14_setoc)/expectation(kde_cr_11_14))



## Calculate occurance of nodes across optimal and Sub-optimal pathways

In [ ]:
s1 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_allpaths.dat')
# s1_index = s1[:,0]
# print(s1_index)
print(s1.shape)
print(s1.shape[0])

Occurence of residues in top-ranked paths:

In [ ]:
def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

tg_source_sink = [1048, 1053, 477, 472, 1192, 1193, 1194, 1195, 1196, 1197] # List of resids of sources and sinks
path_ranks = [0, 1, 2, 3] # ranks of paths
with open('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_nodefreq.dat', 'w') as h:
    resid1 = []
    for i in list(range(s1.shape[0])):
#         if s2[i,5] == 0:
        if s1[i,5] in path_ranks:
            resid1.append(int(s1[i,1]))
    print("shape of residue list:", len(resid1))
    print("No of unique residues in the list:", len(unique(resid1)))
    print("Unique residues are:", unique(resid1))
    for x in unique(resid1):
        if x not in tg_source_sink: ## Excluding the source and sink resids
#             if int(resid1.count(x)) > 10: ## filter only those residues with ocuurance > 10
            print(x, int(resid1.count(x)), file=h)


In [ ]:
! sort -n -k1 /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_nodefreq.dat > /Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_nodefreq_sorted.dat

In [ ]:
s2_tg = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_nodefreq_sorted.dat')

In [ ]:
resid = []
for i in list(range(s2_tg.shape[0])):
        resid.append(str(s2_tg[i,0]))
count = []
for i in list(range(s2_tg.shape[0])):
        count.append((s2_tg[i,1]))

In [ ]:
# print(resid)
# print(count)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
## Plotting
%matplotlib inline

plt.figure(figsize=(14,7))
plt.bar(resid, count, color ='mediumpurple',width = 0.8, label='tgRNA')

# plt.axis([1153, 1200, -0.4, 0.4])

plt.xlabel("Residues", fontname = 'Arial', fontsize = 14, fontweight = 'bold')
plt.ylabel("Occurance in Optimal Paths", fontname = 'Arial', fontsize = 14, fontweight = 'bold')
plt.yticks(fontname = 'Arial', fontsize ='large')
plt.xticks(
    rotation=90, 
    horizontalalignment='center',
    fontweight='light',
    fontsize='large'  
)
plt.legend(prop={"size":14}, frameon=False)
plt.savefig("/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_nodefreq.png", dpi=600)

## Calculate path-length across all optimal and sub-optimal pathways between pair of source-sink

In [ ]:
import numpy as np
s1 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-Antitag/run/Network_Analysis/atgRNA/switchToCatres_allpaths.dat')
s2 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/switchToCatres_allpaths.dat')
s3 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/switchToCatres_allpaths.dat')

d = 0
c = 0
pathlengths_tg = []
pathcount_tg = []
subpathID = [0, 1, 2, 3, 4, 5] ## if want to get lengths of only the optimal path, subpathID = [0]
for j in subpathID:
    for i in list(range(s2.shape[0])):
        if (s2[i,5] == j):
            d += s2[i,3]
            c += 1
        elif d != 0:
                pathlengths_tg.append(d)
                pathcount_tg.append(c)
                d = 0
                c = 0
        else:
            d = 0
            c = 0
print("shape of pathlengths list:", len(pathlengths_tg))
print("Path lengths:", pathcount_tg)
print("Average path length (betweenness estimate, pathlength estimate):", (sum(pathlengths_tg) / len(pathlengths_tg)), (sum(pathcount_tg) / len(pathcount_tg)) )

d = 0
c = 0
pathlengths_atg = []
pathcount_atg = []
for j in subpathID:
    for i in list(range(s1.shape[0])):
        if (s1[i,5] == j):
            d += s1[i,3]
            c += 1
        elif d != 0:
                pathlengths_atg.append(d)
                pathcount_atg.append(c)
                d = 0
                c = 0
        else:
            d = 0
            c = 0
print("shape of pathlengths list:", len(pathlengths_atg))
print("Path lengths:", pathcount_atg)
print("Average path length (betweenness estimate, pathlength estimate):", (sum(pathlengths_atg) / len(pathlengths_atg)), (sum(pathcount_atg) / len(pathcount_atg)))

d = 0
c = 0
pathlengths_cr = []
pathcount_cr = []
for j in subpathID:
    for i in list(range(s3.shape[0])):
        if (s3[i,5] == j):
            d += s3[i,3]
            c += 1
        elif d != 0:
                pathlengths_cr.append(d)
                pathcount_cr.append(c)
                d = 0
                c = 0
        else:
            d = 0
            c = 0

print("shape of pathlengths list:", len(pathlengths_cr))
print("Path lengths:", pathcount_cr)
print("Average path length (betweenness estimate, pathlength estimate):", (sum(pathlengths_cr) / len(pathlengths_cr)), (sum(pathcount_cr) / len(pathcount_cr)))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
## Plotting
%matplotlib inline
fig = plt.figure(figsize=(8,6))

import seaborn as sb
ax = sb.kdeplot(pathcount_cr, color="springgreen", shade=True, alpha=0.5, linewidth=1, label = 'crRNA')
ax = sb.kdeplot(pathcount_tg, color="mediumpurple", shade=True, alpha=0.5, linewidth=1, label = 'tgRNA')
ax = sb.kdeplot(pathcount_atg, color="lightcoral", shade=True, alpha=0.5, linewidth=1, label = 'atgRNA')
# ax = sb.kdeplot(pathlengths_cr, color="springgreen", shade=True, alpha=0.5, linewidth=1, label = 'crRNA')
# ax = sb.kdeplot(pathlengths_tg, color="mediumpurple", shade=True, alpha=0.5, linewidth=1, label = 'tgRNA')
# ax = sb.kdeplot(pathlengths_atg, color="lightcoral", shade=True, alpha=0.5, linewidth=1, label = 'atgRNA')

#Setting the border of the box
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)
# plt.xlim(-2, 6)
# plt.ylim(0, 0.35)


plt.xlabel("Opt Path length", fontname = 'Arial', fontsize = 14, fontweight = 'bold')
plt.ylabel("Density", fontname = 'Arial', fontsize = 14, fontweight = 'bold')
plt.xticks(fontname = 'Arial', fontsize = 12)
plt.yticks(fontname = 'Arial', fontsize = 12)
plt.legend(prop={"size":14}, loc = "upper right",frameon=False)
# plt.legend(prop={"size":14}, frameon=False)
plt.tight_layout()
# plt.savefig("/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_SubOptPathLength_btw.png", dpi=600)